In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [3]:
template = """Answer the users question based only on the following context:

<context>
{context}
</context>

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(temperature=0)

search = DuckDuckGoSearchAPIWrapper()


def retriever(query):
    return search.run(query)

In [4]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [5]:
chain.invoke("What is LangChain?")

'LangChain is a powerful Python framework designed to streamline AI application development, focusing on real-time data processing and integration with Large Language Models (LLMs). It offers features for data communication, generation of vector embeddings, and simplifies the interaction with LLMs, making it efficient for AI developers.'

In [6]:
template = """Provide a better search query for \
web search engine to answer the given question, end \
the queries with ’**’. Question: \
{x} Answer:"""
rewrite_prompt = ChatPromptTemplate.from_template(template)

In [8]:
from langchain import hub

rewrite_prompt = hub.pull("langchain-ai/rewrite")

In [9]:
print(rewrite_prompt.template)


Provide a better search query for web search engine to answer the given question, end the queries with ’**’.  Question {x} Answer:


In [10]:
# Parser to remove the `**`


def _parse(text):
    return text.strip('"').strip("**")

In [11]:
rewriter = rewrite_prompt | ChatOpenAI(temperature=0) | StrOutputParser() | _parse


In [17]:
distracted_query = "man that sam bankman fried trial was crazy! what is langchain?"

rewriter.invoke({"x": distracted_query})


'Who is Sam Bankman-Fried and what is Langchain?'

In [15]:
rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} | rewriter | retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [16]:
rewrite_retrieve_read_chain.invoke(distracted_query)


'I\'m sorry, but there is no information provided in the context about "langchain."'